In [12]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import json
import tabula


In [21]:
def load_data(file_type, file_path):
    if file_type == 'json':
        return pd.read_json(file_path)
    elif file_type == 'xlsx':
        return pd.read_excel(file_path)
    elif file_type == 'html':
        return pd.read_html(file_path)[0]
    elif file_type == 'pdf':
        return tabula.read_pdf(file_path, stream= True, pages='all')[0]
    else:
        raise ValueError("Unsupported file type. Supported types: 'json', 'xlsx', 'html', 'pdf'")


In [25]:
airlines = load_data('json', 'data/airlines.json')
flights = load_data('xlsx', 'data/flights.xlsx')
airports = load_data('xlsx', 'data/airports.xlsx')
planes = load_data('html', 'data/planes.html')
#weather = load_data('pdf', 'data/weather.pdf')

#Format flight data
flights_split = flights.iloc[:, 0].str.split(',', expand=True)

flights_split.columns = ['year', 'month', 'day', 'dep_time', 'sched_dep_time', 'dep_delay', 'arr_time', 'sched_arr_time', 'arr_delay', 'carrier', 'flight', 'tailnum', 'origin', 'dest', 'air_time', 'distance', 'hour', 'minute', 'time_hour']

flights_split = flights_split.apply(pd.to_numeric, errors='ignore')


# Update the flights variable
flights = flights_split

print(flights.dtypes)


C:\Users\carlb\AppData\Local\Temp\ipykernel_6416\1323074342.py:12: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  flights_split = flights_split.apply(pd.to_numeric, errors='ignore')


year               int64
month              int64
day                int64
dep_time          object
sched_dep_time     int64
dep_delay         object
arr_time          object
sched_arr_time     int64
arr_delay         object
carrier           object
flight             int64
tailnum           object
origin            object
dest              object
air_time          object
distance           int64
hour               int64
minute             int64
time_hour         object
dtype: object


In [47]:
# Show types of data
#print('AIRLINES:\n', airlines.dtypes)
#print('\n\nFLIGHTS:\n', flights.dtypes)
# print('\n\nPLANES:\n', planes.dtypes)
# print('\n\nAIRPORTS:\n', airports.dtypes)
#print('\n\nTYPES DES VALEURS DE LA PREMIÈRE LIGNE:\n', flights.iloc[0].apply(type))
#print('\n\nTYPES DES VALEURS DE LA PREMIÈRE LIGNE DES AVIONS:\n', planes.iloc[0].apply(type))
#print('\n\nTYPES DES VALEURS DE LA PREMIÈRE LIGNE DES AÉROPORTS:\n', airports.iloc[0].apply(type))
print('\n\nTYPES DES VALEURS DE LA PREMIÈRE LIGNE DES COMPAGNIES AÉRIENNES:\n', airlines.iloc[0].apply(type))




TYPES DES VALEURS DE LA PREMIÈRE LIGNE DES COMPAGNIES AÉRIENNES:
 carrier    <class 'str'>
name       <class 'str'>
Name: 0, dtype: object


In [30]:
#q1
total_airports = airports['faa'].nunique()
airports_no_dst = airports[airports['dst'] == 'N'].shape[0]
time_zones = airports['tzone'].nunique()
total_airlines = airlines['carrier'].nunique()
total_planes = planes['tailnum'].nunique()

print(f"Total number of airports: {total_airports}")
print(f"Number of airports without DST: {airports_no_dst}")
print(f"Number of time zones: {time_zones}")
print(f"Total number of airlines: {total_airlines}")
print(f"Total number of planes: {total_planes}")


Total number of airports: 1458
Number of airports without DST: 23
Number of time zones: 10
Total number of airlines: 16
Total number of planes: 3322


In [31]:
# 1. Trouver l'aéroport de départ le plus emprunté
most_used_origin = flights['origin'].value_counts().idxmax()
most_used_origin_name = airports[airports['faa'] == most_used_origin]['name'].values[0]
print(f"L'aéroport de départ le plus emprunté est : {most_used_origin_name}")

# 2. Trouver les 10 destinations les plus et les moins prisées
destination_counts = flights['dest'].value_counts()
top_10_destinations = destination_counts.head(10)
bottom_10_destinations = destination_counts.tail(10)

# Ajouter les noms complets des destinations
top_10_destinations = top_10_destinations.reset_index().merge(airports[['faa', 'name']], left_on='index', right_on='faa')
bottom_10_destinations = bottom_10_destinations.reset_index().merge(airports[['faa', 'name']], left_on='index', right_on='faa')

# Calculer les pourcentages
total_flights = flights.shape[0]
top_10_destinations['percentage'] = (top_10_destinations['dest'] / total_flights) * 100
bottom_10_destinations['percentage'] = (bottom_10_destinations['dest'] / total_flights) * 100

print("Les 10 destinations les plus prisées :")
print(top_10_destinations[['name', 'percentage']])

print("Les 10 destinations les moins prisées :")
print(bottom_10_destinations[['name', 'percentage']])

# 3. Trouver les 10 avions qui ont le plus et le moins décollé
plane_counts = flights['tailnum'].value_counts()
top_10_planes = plane_counts.head(10)
bottom_10_planes = plane_counts.tail(10)

print("Les 10 avions qui ont le plus décollé :")
print(top_10_planes)

print("Les 10 avions qui ont le moins décollé :")
print(bottom_10_planes)

L'aéroport de départ le plus emprunté est : Newark Liberty Intl


KeyError: 'index'

In [32]:
# Q8
filtered_flights = flights[flights['carrier'].isin(['UA', 'AA', 'DL'])]

print(filtered_flights.count())

year              104742
month             104742
day               104742
dep_time          104742
sched_dep_time    104742
dep_delay         104742
arr_time          104742
sched_arr_time    104742
arr_delay         104742
carrier           104742
flight            104742
tailnum           104742
origin            104742
dest              104742
air_time          104742
distance          104742
hour              104742
minute            104742
time_hour         104742
dtype: int64
